# **Analiza mehaničkih vibracijskpg sustava s maksimalno 5 stupnjeva slobode gibanja**

## **1. Uvod**
Ovaj Jupyter Notebook omogućava interaktivnu analizu mehaničkih vibracija sustava s više stupnjeva slobode. Koristi **ipywidgets** za interaktivni unos parametara, **NumPy** za računanja, **SciPy** za numeričku integraciju te **Matplotlib** za vizualizaciju odziva sustava.

## **2. Instalacija potrebnih paketa**
Ako nemate instalirane potrebne pakete, instalirajte ih pomoću sljedeće naredbe:
```bash
pip install numpy scipy matplotlib ipywidgets pillow
```

## **3. Uvoz biblioteka**
```python
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, VBox, HBox, Label, Button, Output, ToggleButtons
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from IPython.display import display, Math
from PIL import Image
import os
```

## **4. Globalne varijable**
```python
slider_refs = None  # Globalna varijabla za slidere
output = Output()    # Globalni izlazni objekt za prikaz rezultata
```

## **5. Funkcija za simulaciju sustava**
Ova funkcija:
- Kreira matrice **mase, prigušenja i krutosti**
- Računa **vlastite frekvencije sustava**
- Izračunava **odziv sustava** na različite tipove pobude
- Generira **grafove odziva**
```python
def simulate_system(num_bodies, body_types, masses, springs, dampers, initial_displacements, connection_types, force_type, force_values, omega_values):
    # Implementacija funkcije
```

## **6. Generiranje skice sustava**
Funkcija vizualizira sustav pomoću slika iz vanjskog izvora:
```python
def generate_sketch_with_images(num_bodies, body_types, connection_types, image_folder, force_type):
    # Implementacija funkcije
```

## **7. Izračun ekvivalentnih vrijednosti spojeva**
Ova funkcija računa **ekvivalentne opruge i prigušivače** za različite konfiguracije spojeva:
```python
def calculate_equivalent(k1, k2, c1, c2, connection_type):
    # Implementacija funkcije
```

## **8. Generiranje slidera za unos parametara**
Interaktivni widgeti omogućuju unos vrijednosti za:
- **Krutost opruga**
- **Prigušenje**
- **Sile i frekvencije pobude**
```python
def generate_spring_damper_force_sliders(connection_types, force_type):
    # Implementacija funkcije
```

## **9. Interaktivno podešavanje sustava**
Funkcija omogućava korisniku da:
- **Odabere broj tijela**
- **Postavi početne uvjete**
- **Odabere tip vibracija**
- **Pokrene simulaciju**
```python
def interactive_system(num_bodies):
    # Implementacija funkcije
```

## **10. Pokretanje dinamičkog interaktivnog sučelja**
Pozivom ove funkcije pokreće se cijeli sustav s interaktivnim sučeljem:
```python
def dynamic_interactive_system():
    # Implementacija funkcije

dynamic_interactive_system()
```

## **11. Upute za korištenje**
1. **Odaberite broj tijela u sustavu** pomoću klizača.
2. **Postavite parametre tijela** (masa, tip tijela, itd.).
3. **Definirajte tip veze** između tijela (opruge, prigušivači).
4. **Odaberite tip vibracija** (slobodne ili prisilne).
5. **Pokrenite simulaciju** klikom na gumb "Osvježi".
6. **Pregledajte rezultate**:
   - Vlastite frekvencije sustava
   - Mode shapes (forme vibriranja)
   - Odziv sustava u vremenu
   - Grafove uzbudnih sila

## **12. Zaključak**
Ovaj interaktivni alat omogućava **vizualizaciju i analizu vibracija mehaničkog sustava** pomoću matematičkih modela i numeričkih simulacija. Korištenjem Jupyter Notebook-a, korisnici mogu brzo istraživati utjecaj različitih parametara na dinamiku sustava.


In [4]:
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, VBox, HBox, Label, Button, Output, ToggleButtons
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from IPython.display import display, Math
from PIL import Image
import os

%matplotlib inline

# Globalna varijabla za slidere
slider_refs = None

# Globalni izlazni objekt za prikaz
output = Output()

# Funkcija za simulaciju sustava
def simulate_system(num_bodies, body_types, masses, springs, dampers, initial_displacements, connection_types, force_type, 
                    force_values, omega_values):
    with output:
        output.clear_output()
        # Matrice sustava
        M = np.zeros((num_bodies, num_bodies))  # Matrica mase/inercije
        C = np.zeros((num_bodies, num_bodies))  # Matrica prigušenja
        K = np.zeros((num_bodies, num_bodies))  # Matrica krutosti

        for i in range(num_bodies):
            if body_types[i] == 'Translacijsko':
                M[i, i] = masses[i][0]  # Uzimamo samo prvu masu
            elif body_types[i] == 'Planarno':
                M[i, i] = (3/2) * masses[i][0]  # Translacija i rotacija se zbrajaju; uzimamo samo prvu masu
            elif body_types[i] == 'Kombinirano':
                m_translational = masses[i][0]  # Masa trupa kolica (translational)
                m_planar_1 = (3/2) * masses[i][1]  # Masa prvog kotača (planar)
                m_planar_2 = (3/2) * masses[i][2]  # Masa drugog kotača (planar)

                M[i, i] = m_translational + m_planar_1 + m_planar_2

        # Popuniti matricu prigušenja i krutosti prema pravilima sustava
        for i in range(num_bodies):
            if i == 0:
                if i < len(dampers):
                    C[i, i] += dampers[i]
                if i < len(springs):
                    K[i, i] += springs[i]
            else:
                if i < len(dampers):
                    C[i, i] += dampers[i]
                    C[i, i - 1] -= dampers[i]
                if i < len(springs):
                    K[i, i] += springs[i]
                    K[i, i - 1] -= springs[i]

            if i < num_bodies - 1:
                if i + 1 < len(dampers):
                    C[i, i] += dampers[i + 1]
                    C[i, i + 1] -= dampers[i + 1]
                if i + 1 < len(springs):
                    K[i, i] += springs[i + 1]
                    K[i, i + 1] -= springs[i + 1]
            elif i == num_bodies - 1:
                if i + 1 < len(dampers):
                    C[i, i] += dampers[i + 1]
                if i + 1 < len(springs):
                    K[i, i] += springs[i + 1]

         # Prikaz matričnog zapisa diferencijalne jednadžbe prilagođenog sustavu
        M_str = "\\begin{bmatrix} " + " \\\\ ".join([" & ".join([
            f"m_{{{i+1}}}" if body_types[i] == 'Translacijsko' and i == j else \
            f"\\frac{{3}}{{2}}m_{{{i+1}}}" if body_types[i] == 'Planarno' and i == j else \
            f"m_{{{i+1},1}} + m_{{{i+1},2}} + m_{{{i+1},3}}" if body_types[i] == 'Kombinirano' and i == j else "0"
            for j in range(num_bodies)]) for i in range(num_bodies)]) + " \\end{bmatrix}"

        C_str = "\\begin{bmatrix} " + " \\\\ ".join([" & ".join([f"c_{{{i+1}}}+c_{{{i+2}}}" if i == j else (f"-c_{{{i+2}}}" if j == i + 1 else (f"-c_{{{i+1}}}" if j == i - 1 else "0")) for j in range(num_bodies)]) for i in range(num_bodies)]) + " \\end{bmatrix}"

        K_str = "\\begin{bmatrix} " + " \\\\ ".join([" & ".join([f"k_{{{i+1}}}+k_{{{i+2}}}" if i == j else (f"-k_{{{i+2}}}" if j == i + 1 else (f"-k_{{{i+1}}}" if j == i - 1 else "0")) for j in range(num_bodies)]) for i in range(num_bodies)]) + " \\end{bmatrix}"

        if force_type == 'Slobodne vibracije':
            F_str = "\\begin{bmatrix} " + " \\\\ ".join(["0" for _ in range(num_bodies)]) + " \\end{bmatrix}"
        else:
            F_str = "\\begin{bmatrix} " + " \\\\ ".join([f"F_{{{i+1}}}(t)" for i in range(num_bodies)]) + " \\end{bmatrix}"

        mat_eq = f"{M_str} \\ddot{{X}} + {C_str} \\dot{{X}} + {K_str} X = {F_str}"
        display(Math(mat_eq)) 

        # Izračun vlastitih kružnih frekvencija neprigušenih vibracija
        eigvals, eigvecs = np.linalg.eig(np.linalg.inv(M) @ K)
        omega_n = np.sort(np.sqrt(np.real(eigvals)))  # Neprigušene frekvencije

        # Izračun vlastitih kružnih frekvencija prigušenih vibracija
        damping_ratios = np.diag(C) / (2 * np.sqrt(np.diag(K) * np.diag(M)))
        omega_c = omega_n * np.sqrt(1 - damping_ratios ** 2)  # Prigušene frekvencije

        # Prikaz vlastitih frekvencija u LaTeX formatu kao matrica
        omega_str = "\\begin{bmatrix} " + " & ".join([f"\\omega_{{n,{i+1}}} = {freq:.5f} \\ \\text{{rad/s}}" for i, freq in enumerate(omega_n)]) + " \\\\ " + " & ".join([f"\\omega_{{c,{i+1}}} = {freq:.5f} \\ \\text{{rad/s}}" for i, freq in enumerate(omega_c)]) + " \\end{bmatrix}"
        display(Math(f"\\text{{Vlastite kružne frekvencije: }} {omega_str}"))


        # Prilagodba formi vibriranja s fiksnim točkama na oba kraja za graf
        num_modes = eigvecs.shape[1]
        adjusted_mode_shapes = np.zeros((num_bodies + 2, num_modes))  # Dodajemo dvije fiksne točke (zidove) za graf
        adjusted_mode_shapes[1:-1, :] = eigvecs  # Postavljamo unutarnje pomake za graf

        # Skaliranje formi vibriranja tako da amplituda tijela 1 bude 1
        for i in range(num_modes):
            adjusted_mode_shapes[1:-1, i] /= eigvecs[0, i]

        # Prikaz formi vibriranja u LaTeX formatu (bez fiksnih točaka)
        for i in range(num_modes):
            mode_shape_str = "\\begin{bmatrix} " + " \\\\ ".join([f"{adjusted_mode_shapes[j+1, i]:.5f}" for j in range(num_bodies)]) + " \\end{bmatrix}"
            display(Math(f"\\text{{Glavna forma vibriranja {i+1}: }} {mode_shape_str}"))

        # Grafički prikaz formi vibriranja
        plt.figure(figsize=(10, 6))
        positions = ["Zid"] + [f"Tijelo {i+1}" for i in range(num_bodies)] + ["Zid"]
        # Lista različitih markera
        markers = ['o', 's', '^', 'D', 'v', 'p', '*', 'x', '+']
        for i in range(num_modes):
            marker = markers[i % len(markers)]
            plt.plot(range(len(positions)), adjusted_mode_shapes[:, i], marker=marker, label=f'Glavna forma vibriranja {i+1}')
        plt.axhline(0, color='black', lw=1, ls='--')
        plt.xticks(range(len(positions)), positions)
        plt.xlabel('Položaj (tijelo)')
        plt.ylabel('Amplituda')
        plt.title('Prikaz formi vibriranja sustava')
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.show()



        # Funkcija za izračunavanje derivacija stanja
        def eq_of_motion(t, w):
            X = w[:num_bodies]  # Pozicije
            X_dot = w[num_bodies:]  # Brzine
            force_vector = np.zeros(num_bodies)

            if force_type != 'Slobodne vibracije':
                for i in range(num_bodies):
                    if force_type == 'Fsin(ωt)':
                        force_vector[i] = force_values[i] * np.sin(omega_values[i] * t)
                    elif force_type == 'Fcos(ωt)':
                        force_vector[i] = force_values[i] * np.cos(omega_values[i] * t)

            X_ddot = np.linalg.inv(M) @ (-C @ X_dot - K @ X + force_vector)
            return np.concatenate([X_dot, X_ddot])

        stoptime = 20
        numpoints = 5001
        t_eval = np.linspace(0, stoptime, numpoints)
        x0 = np.zeros(2 * num_bodies)
        x0[:num_bodies] = initial_displacements

        solution = solve_ivp(eq_of_motion, [0, stoptime], x0, t_eval=t_eval)

        display(Math("\\text{Diferencijalne jednadžbe sustava:}"))
        for i in range(num_bodies):
            eq_left = f"{M[i, i]:.2f} \\ddot{{x}}_{i+1}"
            if i == 0:
                eq_left += f" + {dampers[i]:.2f} \\dot{{x}}_{i+1} + {springs[i]:.2f} x_{i+1}"
            else:
                eq_left += f" + {dampers[i]:.2f} (\\dot{{x}}_{i+1} - \\dot{{x}}_{i})"
                eq_left += f" + {springs[i]:.2f} (x_{i+1} - x_{i})"
            if i < num_bodies - 1:
                eq_left += f" + {dampers[i+1]:.2f} (\\dot{{x}}_{i+1} - \\dot{{x}}_{i+2})"
                eq_left += f" + {springs[i+1]:.2f} (x_{i+1} - x_{i+2})"
            elif i == num_bodies - 1:
                eq_left += f" + {dampers[i+1]:.2f} \\dot{{x}}_{i+1} + {springs[i+1]:.2f} x_{i+1}"
                
            eq_right = "0"
            if force_type != 'Slobodne vibracije':
                if force_type == 'Fsin(ωt)':
                    eq_right = f"{force_values[i]:.2f} \\sin({omega_values[i]:.2f} t)"
                elif force_type == 'Fcos(ωt)':
                    eq_right = f"{force_values[i]:.2f} \\cos({omega_values[i]:.2f} t)"

            display(Math(f"{eq_left} = {eq_right}"))

        

        # Prikaz odziva svakog tijela uz sve uzbudne sile, koristeći sekundarnu os za sile
        colors = ['r', 'g', 'm', 'c', 'y']  # Boje za sile

        for i in range(num_bodies):
            fig, ax1 = plt.subplots(figsize=(9, 6))

            # Prva y-os (odziv tijela)
            ax1.set_xlabel('Vrijeme (s)')
            ax1.set_ylabel('Pomak (m)', color='b')
            ax1.plot(solution.t, solution.y[i], label=rf'$x_{i+1}$ (m)', linewidth=2, color='b')
            ax1.tick_params(axis='y', labelcolor='b')

            # Druga y-os (uzbudne sile)
            ax2 = ax1.twinx()
            ax2.set_ylabel('Sila (N)', color='r')

            if force_type != 'Slobodne vibracije':
                for j in range(num_bodies):  # Svi izvori sila prikazuju se na svakom grafu
                    if force_values[j] > 0:
                        if force_type == 'Fsin(ωt)':
                            force_t = force_values[j] * np.sin(omega_values[j] * t_eval)
                        elif force_type == 'Fcos(ωt)':
                            force_t = force_values[j] * np.cos(omega_values[j] * t_eval)

                        # Sile ostavljamo u pravoj amplitudi bez normalizacije
                        ax2.plot(t_eval, force_t, linestyle='--', color=colors[j % len(colors)], label=f'Sila {j+1} (N)')

            # Postavljanje oznaka i prikaza
            ax1.set_title(f'Dinamički odziv i uzbudne sile - Tijelo {i+1}', fontsize=16)
            ax1.grid(True, linestyle=':')
            
            # Kombinirana legenda za oba y-osa
            lines, labels = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines + lines2, labels + labels2, fontsize=12, loc='upper right')

            plt.show()
        

# Funkcija za generiranje skice sustava prema korisničkim parametrima koristeći slike iz vanjskog izvora
def generate_sketch_with_images(num_bodies, body_types, connection_types, image_folder, force_type):
    fig, ax = plt.subplots(figsize=(15, 7.5))
    ax.set_xlim(-1.5, num_bodies * 3 + 2)
    ax.set_ylim(-2, 2)

    # Prikaz lijevog zida
    left_wall_image = Image.open(os.path.join(image_folder, 'lijevi_zid.png'))
    ax.imshow(left_wall_image, extent=(-1.5, -1, -1.5, 1.5), zorder=1)

    # Prikaz prve veze
    first_connection_type = connection_types[0]
    first_connection_image_path = os.path.join(image_folder, f'{first_connection_type.lower().replace(" ", "_")}.png')
    if os.path.exists(first_connection_image_path):
        first_connection_image = Image.open(first_connection_image_path)
        ax.imshow(first_connection_image, extent=(-1, 1.5, -0.5, 0.5), zorder=2)

    x_pos = 0.5

    for i in range(num_bodies):
        # Učitavanje slike tijela na osnovu tipa
        body_image_path = os.path.join(image_folder, f'{body_types[i].lower()}.png')
        if os.path.exists(body_image_path):
            body_image = Image.open(body_image_path)
            ax.imshow(body_image, extent=(x_pos, x_pos + 2, -1, 1), zorder=1)
            ax.text(x_pos + 1, -1.5, f'Tijelo {i + 1}\n({body_types[i]})', ha='center', va='center')

        # Prikaz sila ako je odabrana tip sila
        if force_type in ['Fsin(ωt)', 'Fcos(ωt)']:
            ax.arrow(x_pos + 0.5, 1.25, 1, 0, head_width=0.1, head_length=0.2, fc='black', ec='black', zorder=3)
            ax.text(x_pos + 1, 1.5, f'F{i + 1}(t)', ha='center', zorder=3)
        
        # Prikaz veza između tijela i zidova koristeći slike
        if i < num_bodies - 1:
            x_connection = x_pos + 1
            connection_type = connection_types[i + 1]
            connection_image_path = os.path.join(image_folder, f'{connection_type.lower().replace(" ", "_")}.png')
            if os.path.exists(connection_image_path):
                connection_image = Image.open(connection_image_path)
                ax.imshow(connection_image, extent=(x_connection, x_connection + 2.5, -0.5, 0.5), zorder=2)

        x_pos += 2.5


    # Prikaz posljednje veze
    last_connection_type = connection_types[-1]
    last_connection_image_path = os.path.join(image_folder, f'{last_connection_type.lower().replace(" ", "_")}.png')
    if os.path.exists(last_connection_image_path):
        last_connection_image = Image.open(last_connection_image_path)
        ax.imshow(last_connection_image, extent=(x_pos - 1.5, x_pos + 1, -0.5, 0.5), zorder=2)
        
    # Prikaz desnog zida
    right_wall_image = Image.open(os.path.join(image_folder, 'desni_zid.png'))
    ax.imshow(right_wall_image, extent=(x_pos + 1, x_pos + 1.5, -1.5, 1.5), zorder=1)

    ax.axis('off')
    plt.show()


# Funkcija za izračunavanje ekvivalentnih vrijednosti
def calculate_equivalent(k1, k2, c1, c2, connection_type):
    k_eq = 0
    c_eq = 0

    if connection_type == '2 Opruge serijski':
        k_eq = 1 / (1/k1 + 1/k2)
    elif connection_type == '2 Opruge paralelno':
        k_eq = k1 + k2
    elif connection_type == '2 Prigušivača paralelno':
        c_eq = c1 + c2
    elif connection_type == 'Opruga i Prigušivač paralelno':
        k_eq = k1
        c_eq = c1
    elif connection_type == 'Samo Opruga':
        k_eq = k1
        c_eq = 0
    elif connection_type == 'Samo Prigušivač':
        k_eq = 0
        c_eq = c1
    elif connection_type == '2 Prigušivača serijski':
        c_eq = 1 / (1/c1 + 1/c2)

    return k_eq, c_eq

# Funkcija za generiranje slidera za opruge, prigušivače i sile
def generate_spring_damper_force_sliders(connection_types, force_type):
    global slider_refs
    if slider_refs is None:
        slider_refs = {}
    sliders_box = []

    for i, conn in enumerate(connection_types):
        if "Oprug" in conn:
            num_springs = 2 if "2" in conn else 1
            for j in range(num_springs):
                key = f'k{i+1}{j+1}'
                slider = slider_refs.get(key, FloatSlider(value=100.0, min=0.0, max=2000.0, step=50.0, description=f'Opruga {i+1}_{j+1} (N/m)'))
                sliders_box.append(slider)
                slider_refs[key] = slider

        if "Prigušivač" in conn:
            num_dampers = 2 if "2" in conn else 1
            for j in range(num_dampers):
                key = f'c{i+1}{j+1}'
                slider = slider_refs.get(key, FloatSlider(value=0.5, min=0.0, max=30.0, step=0.2, description=f'Prigušivač {i+1}_{j+1} (Ns/m)'))
                sliders_box.append(slider)
                slider_refs[key] = slider

    if force_type != 'Slobodne vibracije':
        for i in range(len(connection_types) - 1):
            force_key = f'F{i+1}'
            omega_key = f'ω{i+1}'
            force_slider = slider_refs.get(force_key, FloatSlider(value=0.0, min=0.0, max=100.0, step=1.0, description=f'Sila {i+1} (N)'))
            omega_slider = slider_refs.get(omega_key, FloatSlider(value=1.0, min=0.1, max=100.0, step=0.1, description=f'ω {i+1} (rad/s)'))
            sliders_box.append(force_slider)
            sliders_box.append(omega_slider)
            slider_refs[force_key] = force_slider
            slider_refs[omega_key] = omega_slider

    return VBox(sliders_box)

# Funkcija za ažuriranje sustava prema odabiru korisnika
def interactive_system(num_bodies):
    num_bodies_slider = IntSlider(value=num_bodies, min=1, max=5, step=1, description="Broj tijela")

    body_type_options = ['Translacijsko', 'Planarno', 'Kombinirano']

    mass_sliders = []

    def update_mass_sliders():
        mass_sliders.clear()
        for i in range(num_bodies):
            sliders = []
            for j in range(3):
                sliders.append(FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description=f'Masa {i+1}_{j+1} (kg)'))
            mass_sliders.append(sliders)

    def create_body_type_slider(i):
        slider = Dropdown(options=body_type_options, value='Translacijsko', description=f'Tip {i+1}')
        slider.observe(lambda change: update_mass_sliders(), names='value')
        return slider

    body_type_sliders = [create_body_type_slider(i) for i in range(num_bodies)]

    update_mass_sliders()

    connection_sliders = [
        Dropdown(options=[
            '2 Opruge serijski', '2 Opruge paralelno',
            '2 Prigušivača paralelno',
            '2 Prigušivača serijski',
            'Opruga i Prigušivač paralelno',
            'Samo Opruga', 'Samo Prigušivač'],
            value='2 Opruge paralelno', description=f'Veza {i+1}')
        for i in range(num_bodies + 1)
    ]

    initial_displacement_sliders = [
        FloatSlider(value=0.5 if i % 2 == 0 else -0.5, min=-1.0, max=1.0, step=0.01, description=f'Početni pomak {i+1} (m)')
        for i in range(num_bodies)
    ]

    force_type_slider = ToggleButtons(
        options=['Slobodne vibracije', 'Fsin(ωt)', 'Fcos(ωt)'],
        description='Tip vibracija:',
        value='Slobodne vibracije'
    )

    def update_system(**kwargs):
        global slider_refs
        global output

        num_bodies = kwargs['num_bodies']
        body_types = [kwargs[f'Tip {i+1}'] for i in range(num_bodies)]
        masses = []
        for i in range(num_bodies):
            sliders = [kwargs[f'Masa {i+1}_{j+1}'] for j in range(3)]
            masses.append(sliders)

        initial_displacements = [kwargs[f'Početni pomak {i+1}'] for i in range(num_bodies)]
        connection_types = [kwargs[f'Veza {i+1}'] for i in range(num_bodies + 1)]
        force_type = kwargs['Tip vibracija:']
        
        # Generiranje skice sustava koristeći slike iz vanjskog izvora
        image_folder = r"D:\Faks\Zavrsni rad"  # Putanja do foldera sa slikama
        generate_sketch_with_images(num_bodies, body_types, connection_types, image_folder, force_type)

        sliders_box = generate_spring_damper_force_sliders(connection_types, force_type)
        display(sliders_box)

        springs = []
        dampers = []
        for i, conn in enumerate(connection_types):
            k1 = slider_refs.get(f'k{i+1}1', FloatSlider(value=100.0)).value
            k2 = slider_refs.get(f'k{i+1}2', FloatSlider(value=100.0)).value
            c1 = slider_refs.get(f'c{i+1}1', FloatSlider(value=0.5)).value
            c2 = slider_refs.get(f'c{i+1}2', FloatSlider(value=0.5)).value

            k_eq, c_eq = calculate_equivalent(k1, k2, c1, c2, conn)
            springs.append(k_eq)
            dampers.append(c_eq)
        force_values = [slider.value for slider in sliders_box.children if 'Sila' in slider.description]
        omega_values = [slider.value for slider in sliders_box.children if 'ω' in slider.description]

        simulate_system(num_bodies, body_types, masses, springs, dampers, initial_displacements, connection_types, force_type, force_values, omega_values)

    refresh_button = Button(description="Osvježi", button_style='success')
    refresh_button.on_click(lambda _: update_system(**{key: control.value for key, control in controls.items()}))

    controls = {
        'num_bodies': num_bodies_slider,
        'Tip vibracija:': force_type_slider
    }
    controls.update({f'Tip {i+1}': body_type_sliders[i] for i in range(num_bodies)})
    for i, slider_group in enumerate(mass_sliders):
        controls.update({f'Masa {i+1}_{j+1}': slider for j, slider in enumerate(slider_group)})

    controls.update({f'Početni pomak {i+1}': initial_displacement_sliders[i] for i in range(num_bodies)})
    controls.update({f'Veza {i+1}': connection_sliders[i] for i in range(num_bodies + 1)})
    interact(update_system, **controls)

    display(VBox([HBox([refresh_button]), output]))

# Pokretanje sustava
def dynamic_interactive_system():
    num_bodies_slider = IntSlider(value=3, min=1, max=5, step=1, description="Broj tijela")

    def update_num_bodies(num_bodies):
        interactive_system(num_bodies)

    interact(update_num_bodies, num_bodies=num_bodies_slider)

dynamic_interactive_system()



interactive(children=(IntSlider(value=3, description='Broj tijela', max=5, min=1), Output()), _dom_classes=('w…